## 학습정리

### 19장. 딥러닝
* 딥러닝(deep learning) : 한 개이상의 은닉층을 지닌 '깊은' 신경망을 의미
    * 일반적으로 다양한 신경망 구조를 모두 아우는 용어
####  19.1 텐서
* 복잡한 신경망 구조를 다루다 보면 더 고차원의 배열이 필요
* 텐서(tensor) : n차원의 배열 (대부분의 신경망 라이브러리의 명칭 따라서 텐서라 함, 수학적으로 따지면 텐서로 부르면 안된다는 것 인지)
    * 이 책에서는 간단하게 텐서를 리스트로 구현
        * 벡터, 행렬, 고차원의 텐서는 리스트의 일종으로 볼 수 있지만, 리스트는 n차원의 배열이 아님
    
####  19.2 층 추상화
* 딥러닝에서는 다양한 신경망 구조를 사용가능
    * ex. 각 뉴런이 이전 입력값을 기억하도록 만들거나 시그모이드 대신 다른 활성화 함수를 사용
    * 두 층보다 더 깊은 신경망생성
    * 신경망의 각 층을 나타내는 Layer 추상화 : 입력값에 특정 함수를 적용하거나 역전파를 할 수 있어야 함
    * 18장에서 구현한 신경망 : 선형 층위에 시그모이드 층으로 구성된 신경망
####  19.3 선형 층
* 18장에서 구현한 신경망을 기반으로 뉴런의 dot(weights, inputs)를 나타내는 선형 층(linear layer)을 구현 가능
    * 선형 층의 초기값 : 임의로 생성
        * 초깃값의 설정은 매우 중요. 파라미터의 초깃값에 따라 신경망의 학습속도,학습 여부 자체가 결정
        * 만약 초깃값이 너무 크면 활성화 함수의 그래디언트는 0에 가깝게 됨
            * 그래디언트가 0에 가까운 파라미터는 경사하강법으로 학습 불가
        * 파라미터를 임의로 생성해 주는 방법
            1. 균등 분포 : random.random(), 0과 1사이 임의의 값으로 설정
            2. 표준 정규 분포 : 임의의 초깃값 생성
            3. Xavier 초기화 : 평균이 0이고 편차가 2/(입력 값의 개수 + 출력 값의 개수)인 정규 분포에서 임의의 초깃값 생성
            
####  19.4 순차적인 층으로 구성된 신경망
* 신경망을 순차적인 층으로 구성되어 있다고 생각
    * 여러 층을 하나의 층으로 표현 가능 : 하나의 신경망 자체를 Layer메서드로 하나의 층으로 표현
    
####  19.5 손실 함수와 최적화
* 손실 함수(loss function)와 그래디언트 함수 

####  19.6 예시 : XOR문제 다시 풀어 보기
####  19.7 다른 활성화 함수
####  19.8 예시 : Fizz Buzz 다시 풀어보기
####  19.9 Softmax와 Cross-Entropy
####  19.10 드롭아웃
####  19.11 예시 : MNIST
####  19.12 모델 저장 및 불러오기
#### 19.13 더 공부해 보고 싶다면

## code

In [2]:
import numpy as np
import pandas as pd

In [3]:
import math

In [4]:
def normal_cdf(x : float, mu: float = 0, sigma: float = 1) -> float :
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma )) / 2

def inverse_normal_cdf(p : float,
                      mu : float = 1,
                      sigma : float = 1,
                      tolerance : float = 0.00001) -> float :
    """이진 검색을 사용해서 역함수를 근사"""
    
    # 표준정규분포가 아니라면 표준정규분포로 변환
    if mu != 0 or sigma != 1:
        return mu + sigma*inverse_normal_cdf(p, tolerance=tolerance)
    
    low_z = -10.0        # normal_cdf(-10)은 0에 근접
    hi_z = 10.0          # normal_cdf(10)은 0에 근접
    while hi_z - low_z > tolerance :
        mid_z = (low_z + hi_z) / 2  # 중간 값
        mid_p = normal_cdf(mid_z)   # 중간 값의 누적분포 값 계산
        if mid_p < p :
            low_z = mid_z           # 중간 값이 너무 작다면 더 큰 값들을 검색
        else :
            hi_z = mid_z            # 중간 값이 너무 크다면 더 작은 값들을 검색
    
    return mid_z

In [5]:
# 텐서를 리스트로 정의
Tensor = list

# 텐서의 크기를 반환해 주는 도우미 함수 
from typing import List

def shape(tensor : Tensor) -> List[int] :
    sizes : List[int] = []
    while isinstance(tensor, list) :
        sizes.append(len(tensor))
        tensor = tensor[0]
    return sizes

In [6]:
print(shape([1,2,3]))
print(np.shape([1,2,3]))

[3]
(3,)


In [7]:
print(shape([[1,2],[3,4],[5,6]]))
print(np.shape([[1,2],[3,4],[5,6]]))

[3, 2]
(3, 2)


In [8]:
def is_1d(tensor: Tensor) -> bool :
    """만약 tensor[0]이 리스트라면 고차원 텐서를 의미
    그러지 않으면 1차원 벡터를 의미"""
    return not isinstance(tensor[0], list)

In [9]:
is_1d([1,2,3]) # 1차원 벡터

True

In [10]:
def tensor_sum(tensor: Tensor) -> float :
    """텐서 안의 모든 값의 합을 반환"""
    if is_1d(tensor) :
        return sum(tensor) # 벡터의 경우, 파이썬의 기본 함수인 sum을 사용
    else :
        return sum(tensor_sum(tensor_i) for tensor_i in tensor)
    

In [11]:
print(tensor_sum([1,2,3]))
print(np.sum([1,2,3]))

6
6


In [12]:
print(tensor_sum([[1,2],[3,4]]))
print(np.sum([[1,2],[3,4]]))

10
10


In [13]:
from typing import Callable
def tensor_apply(f: Callable[[float], float], tensor : Tensor) -> Tensor :
    """텐서 안의 모든 값에 f를 적용"""
    if is_1d(tensor) :
        return [f(x) for x in tensor]
    else :
        return [tensor_apply(f, tensor_i) for tensor_i in tensor]

In [14]:
print(tensor_apply(lambda x : x+1, [1,2,3]))
print(tensor_apply(lambda x : 2*x, [[1,2],[3,4]]))

[2, 3, 4]
[[2, 4], [6, 8]]


In [15]:
# 크기와 동일한 0텐서를 만듬
def zeros_like(tensor: Tensor) -> Tensor :
    return tensor_apply(lambda _ : 0.0, tensor)

In [16]:
print(zeros_like([1,2,3]))
print(zeros_like([[1,2],[3,4]]))

[0.0, 0.0, 0.0]
[[0.0, 0.0], [0.0, 0.0]]


In [17]:
# 두 텐서의 대칭되는 값에 일괄적으로 함수를 적용할 수 있게 해주는 함수
def tensor_combine(f: Callable[[float, float], float],
                  t1: Tensor,
                  t2: Tensor) -> Tensor :
    """두 텐서의 대칭되는 t1과 t2에 일괄적으로 함수를 적용"""
    if is_1d(t1) :
        return [f(x,y) for x,y in zip(t1, t2)]
    else :
        return [tensor_combine(f, t1_i, t2_i) for t1_i, t2_i in zip(t1,t2)]


In [18]:
import operator
print(tensor_combine(operator.add, [1,2,3], [4,5,6]))
print(tensor_combine(operator.mul, [1,2,3], [4,5,6]))

[5, 7, 9]
[4, 10, 18]


In [19]:
# 층 추상화
from typing import Iterable,Tuple

class Layer :
    
    """딥러닝 신경망은 Layer들로 구성되어 있음
    각 Layer별로 순방향으로 입력값에 어떤 계산을 하고 역방향으로 그래디언트를 전파해야함"""
    
    def forward(self, input) :
        """
        타입이 명시되어 있지 않은 것을 유의
        입력층과 출력값의 타입을 제한하지 않음
        """
        raise NotImplementedError
    
    def backward(self, gradient) :
        """
        역방향에서도 그래디언트의 타입을 제한하지 않을 것
        메서드를 호출할 때 유의
        """
        raise NotImplementedError
    
    def params(self) -> Iterable[Tensor] :
        """
        해당 층의 파라미터를 반환
        기본적으로 아무것도 반환하지 않음
        만약 특정 층에서 반환할 파라미터가 없다면 구현할 필요 없음
        """
        return ()
    
    def grads(self) -> Iterable[Tensor] :
        """
        params()처럼 그래디언트를 반환
        """
        return ()
    

In [20]:
# 시그모이드 층 구현
def sigmoid(t: float) -> float :
    return 1/ (1 + math.exp(-t))

class Sigmoid(Layer) :
    def forward(self, input: Tensor) -> Tensor :
        """
        입력된 Tensor의 모든 값에 sigmoid를 계산
        backpropagation을 위해 결괏값을 저장
        """
        self.sigmoids = tensor_apply(sigmoid, input)
        return self.sigmoids
    
    def backward(self, gradient: Tensor) -> Tensor:
        return tensor_combine(lambda sig, 
                              grad: sig*(1-sig)*grad, # 미적분과 연쇄법칙 : output*(1-output)*(output-target) 
                              self.sigmoids, gradient) 


In [21]:
# 선형층에서의 파라미터 생성
import random
from scipy.stats import norm

def random_uniform(*dims: int) -> Tensor :
    if len(dims) == 1 :
        return [random.random() for _ in range(dims[0])]
    else :
        return [random_uniform(*dims[1:]) for _ in range(dims[0])]
    
def random_normal(*dims: int,
                 mean : float = 0.0,
                 variance : float = 1.0) -> Tensor :
    if len(dims) == 1 :
        return [mean + variance * inverse_normal_cdf(random.random()) for _ in range(dims[0])]
    
    else:
        return [random_normal(*dims[1:], mean = mean, variance = variance) for _ in range(dims[0])]
    

In [22]:
shape(random_uniform(2,3,4))

[2, 3, 4]

In [23]:
import sys
sys.setrecursionlimit(10000)

In [24]:
shape(random_normal(5,6,mean=10))

RecursionError: maximum recursion depth exceeded in comparison

In [ ]:
# 19.5 손실 함수와 최적화

class Loss :
    def loss(self, predicted: Tensor, actual: Tensor) -> float:
        """예측 값이 얼마나 정확한가? (손실값이 크면 클수록 좋지 않음)"""
        raise NotImplementedError
        
    def gradient(self, predicted: Tensor, actual: Tensor) -> Tensor :
        """예측값이 변하면 손실은 얼마나 변하는가? """
        raise NotImplementedError

# 평균제곱 오차 손실함수 
class SSE(loss) :
    def loss(self, predicted: Tensor, actual: Tensor) -> float :
        # 각 예측값의 제곱오차를 계산한 수 텐서로 표현
        squared_errors = tensor_combine(
            lambda predicted, actual: (predicted - actual)**2, predicted, actual)
        
        # 모든 제곱오차를 더함
        return tensor_sum(squared_errors)
    
    def gradient(self, predicted: Tensor, actual: Tensor) -> Tensor :
        return tensor_combine(
            lambda predicted, actual: 2*(predicted - actual),
            predicted,
            actual)
    